In [1]:
import aws_stepfunction as sfn
from boto_session_manager import BotoSesManager
from s3pathlib import S3Path, context
from rich import print as rprint

bsm = BotoSesManager(profile_name="aws_data_lab_sanhe_us_east_1")
context.attach_boto_session(bsm.boto_ses)

In [2]:
bucket = f"{bsm.aws_account_id}-{bsm.aws_region}-data"
s3dir_root = S3Path(bucket, "projects/aws_stepfunction/examples/map-pattern/").to_dir()


In [4]:
def prepare_data():
    s3dir_source = s3dir_root.joinpath("source")
    print(f"preview source data at: {s3dir_source.console_url}")
    n_file = 3
    for ith in range(1, 1+n_file):
        s3path = s3dir_source / f"{ith}.txt"
        s3path.write_text(f"this is the {ith} th file")

prepare_data()

preview source data at: https://console.aws.amazon.com/s3/object/669508176277-us-east-1-data?prefix=projects/aws_stepfunction/examples/map-pattern/source


In [7]:
task1_coordinator = sfn.LambdaTask(
    id="Task1-Coordinator",
    lbd_func_name="aws_stepfunction_map_pattern-task1_coordinator",
    lbd_package="worker.py",
    lbd_handler="worker.lambda_handler",
    lbd_aws_account_id=bsm.aws_account_id,
    lbd_aws_region=bsm.aws_region,
    lbd_code_s3_bucket=bucket,
)

task2_worker = sfn.LambdaTask(
    id="Task2-Worker",
    lbd_func_name="aws_stepfunction_map_pattern-task1_worker",
    lbd_package="worker.py",
    lbd_handler="worker.lambda_handler",
    lbd_aws_account_id=bsm.aws_account_id,
    lbd_aws_region=bsm.aws_region,
    lbd_code_s3_bucket=bucket,
)

In [8]:
workflow = sfn.Workflow()
(
    workflow.start_from(task1_coordinator)
    .map(
        (
            workflow.subflow_from(task2_worker)
            .end()
        )
    )
    .end()
)
rprint(workflow.serialize())

{
    'StartAt': 'Task1-Coordinator',
    'States': {
        'Task1-Coordinator': {
            'Type': 'Task',
            'Resource': 'arn:aws:states:::lambda:invoke',
            'Next': 'Map-after-Task1-Coordinator',
            'Parameters': {
                'Payload.$': '$',
                'FunctionName': 
'arn:aws:lambda:us-east-1:669508176277:function:aws_stepfunction_map_pattern-task1_coordinator'
            },
            'OutputPath': '$.Payload',
            'Retry': [
                {
                    'ErrorEquals': [
                        'Lambda.ServiceException',
                        'Lambda.AWSLambdaException',
                        'Lambda.SdkClientException'
                    ],
                    'IntervalSeconds': 2,
                    'BackoffRate': 2,
                    'MaxAttempts': 3
                }
            ]
        },
        'Map-after-Task1-Coordinator': {
            'Type': 'Map',
            'Iterator': {
                'StartAt': 'Task2-Worker',
                'States': {
                    'Task2-Worker': {
                        'Type': 'Task',
                        'Resource': 'arn:aws:states:::lambda:invoke',
                        'End': True,
                        'Parameters': {
                            'Payload.$': '$',
                            'FunctionName': 
'arn:aws:lambda:us-east-1:669508176277:function:aws_stepfunction_map_pattern-task1_worker'
                        },
                        'OutputPath': '$.Payload',
                        'Retry': [
                            {
                                'ErrorEquals': [
                                    'Lambda.ServiceException',
                                    'Lambda.AWSLambdaException',
                                    'Lambda.SdkClientException'
                                ],
                                'IntervalSeconds': 2,
                                'BackoffRate': 2,
                                'MaxAttempts': 3
                            }
                        ]
                    }
                }
            },
            'End': True
        }
    }
}

In [10]:
sfn_name = "aws_stepfunction_map_pattern"

state_machine = sfn.StateMachine(
    name=sfn_name,
    workflow=workflow,
    role_arn="arn:aws:iam::669508176277:role/sanhe-for-everything-admin",
)

In [11]:
state_machine.deploy(bsm=bsm)

detect whether the magic task is used ...
    yes
identify necessary S3 bucket and IAM role ...


ClientError: An error occurred (NoSuchTagSet) when calling the GetBucketTagging operation: The TagSet does not exist